In [1]:
import time
import requests
import lxml.html
import re

In [7]:
def main():
    session=requests.Session()
    response = session.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
    urls = scrape_list_page(response)
    for url in urls:
        time.sleep(1)
        response = session.get(url)
        ebook = scrape_detail_page(response)
        print(ebook)
        break
        

In [8]:
def scrape_list_page(response):
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)
    for a in root.cssselect('.view_box .book_tit a'):
        url = a.get('href')
        yield url

In [9]:
def scrape_detail_page(response):
    root = lxml.html.fromstring(response.content)
    ebook = {
        'url' : response.url,
        'title' : root.cssselect('.store_product_info_box h3')[0].text_content(),
        'price' : root.cssselect('.pbr strong')[0].text_content(),
        'content' : [normalize_spaces(p.text_content())
                    for p in root.cssselect('#tabs_3 .hanbit_edit_view p')
                    if normalize_spaces(p.text_content()) != '']
    }
    return ebook

In [12]:
def normalize_spaces(s):
    return re.sub(r'\s+', ' ', s).strip()

In [13]:
if __name__ == '__main__':
    main()

{'url': 'https://www.hanbit.co.kr/store/books/look.php?p_code=B3178834938', 'title': '받침 없는 한글 동화 : 4권 세트', 'price': '49,680', 'content': ['<1편>', '이렇게 읽어요', '1. 무시무시 마녀가 이사 와!', '2. 아기 고래 뿌우의 노래', '<2편>', '이렇게 읽어요', '1. 도, 도, 도깨비다!', '2. 느티나무 아파트', '<3편>', '이렇게 읽어요', '1. 바쁘다 바빠 너구리 바빠', '2. 꼬마 지우개 마구마구', '<4편>', '이렇게 읽어요', '1. 도깨비 파자마 파티', '2. 꼬마 두루미 뚜루']}
